# Recurrent DQN

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

# Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
# import numpy as np
# state = env.reset()
# for _ in range(10):
#     # env.render()
#     action = env.action_space.sample()
#     next_state, reward, done, info = env.step(action) # take a random action
#     #print('state, action, next_state, reward, done, info:', state, action, next_state, reward, done, info)
#     state = next_state
#     if done:
#         state = env.reset()

To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [4]:
def model_input(state_size, hidden_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # RNN
    cell = tf.nn.rnn_cell.GRUCell(hidden_size)
    #cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell], state_is_tuple=False)
    initial_state = cells.zero_state(batch_size, tf.float32)
    is_training = tf.placeholder(dtype=tf.bool, shape=[], name='is_training')
    print('initial_state, cells, cell', initial_state, cells, cell)
    return states, actions, targetQs, cells, initial_state, is_training

In [5]:
def init_xavier(random_seed=1, dtype=tf.float32, uniform=False):
    xavier = tf.contrib.layers.xavier_initializer(
        dtype=dtype,
        seed=tf.set_random_seed(random_seed), 
        uniform=uniform) # False: normal
    return xavier

In [6]:
def mlp(inputs, units, trainable=True):
    outputs = tf.layers.dense(
        inputs=inputs,
        units=units,
        activation=None,
        use_bias=True,
        kernel_initializer=init_xavier(), # Xavier with normal init
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=trainable,
        name=None,
        reuse=None)
    return outputs

In [7]:
# tf.nn.leaky_relu(
#     features,
#     alpha=0.2,
#     name=None
# )
def nl(inputs, alpha=0.2):
    outputs = tf.maximum(alpha * inputs, inputs)
    return outputs

In [10]:
def bn(inputs, is_training=False):
    outputs = tf.layers.batch_normalization(
        inputs=inputs,
        axis=-1,
        momentum=0.99,
        epsilon=0.001,
        center=True,
        scale=True,
        beta_initializer=tf.zeros_initializer(),
        gamma_initializer=tf.ones_initializer(),
        moving_mean_initializer=tf.zeros_initializer(),
        moving_variance_initializer=tf.ones_initializer(),
        beta_regularizer=None,
        gamma_regularizer=None,
        beta_constraint=None,
        gamma_constraint=None,
        training=is_training,
        trainable=True,
        name=None,
        reuse=None,
        renorm=False,
        renorm_clipping=None,
        renorm_momentum=0.99,
        fused=None,
        virtual_batch_size=None,
        adjustment=None)
    return outputs

In [17]:
# Actor-Critic or Discriminator/classifier or Q-Net/Q-function
def D(states, action_size, initial_state, cells, hidden_size, reuse=False, is_training=False): 
    with tf.variable_scope('D', reuse=reuse):
        # First fully connected layer
        inputs = mlp(inputs=states, units=hidden_size)
        inputs = bn(inputs=inputs, is_training=is_training)
        inputs = nl(inputs)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size and
        # static means can NOT adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, hidden_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cells, inputs=inputs_rnn, 
                                                     initial_state=initial_state)
        #outputs_rnn = tf.layers.batch_normalization(inputs=outputs_rnn, training=is_training)
        #final_state = tf.layers.batch_normalization(inputs=final_state, training=is_training)
        print(outputs_rnn.shape, final_state)
        outputs = tf.reshape(outputs_rnn, [-1, hidden_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        actions_logits = mlp(inputs=outputs, units=action_size)
        print(actions_logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return actions_logits, final_state

In [18]:
def model_loss(action_size, hidden_size, states, cells, initial_state, actions, targetQs, is_training=False):
    
    actions_logits, final_state = D(states=states, cells=cells, initial_state=initial_state, 
                                    hidden_size=hidden_size, action_size=action_size, is_training=is_training)
    
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    
    loss = tf.reduce_mean((Qs - targetQs)**2)
    
    return actions_logits, final_state, loss

In [19]:
def model_opt(loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('D')]

    # # Optimize MLP/CNN/RNN without clipping grads
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=d_vars)

        # Optimize RNN
        #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
        #grads = tf.gradients(loss, g_vars)
        #opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))
    return opt

In [20]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        (self.states, self.actions, self.targetQs, cells, self.initial_state, \
         self.is_training) = model_input(state_size=state_size, hidden_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, cells=cells, initial_state=self.initial_state, 
            is_training=self.is_training)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [21]:
# Network parameters
action_size = 2
state_size = 4
hidden_size = 128               # number of units in each Q-network hidden layer
learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
# memory_size = 128            # memory capacity - 1000 DQN
batch_size = 128             # experience mini-batch size - 20 DQN
gamma = 0.99                 # future reward discount

In [22]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# # Init the memory
# memory = Memory(max_size=batch_size)

initial_state, cells, cell Tensor("MultiRNNCellZeroState/MultiRNNCellZeroState/zeros:0", shape=(1, 128), dtype=float32) <tensorflow.python.ops.rnn_cell_impl.MultiRNNCell object at 0x7fa4101f44a8> <tensorflow.python.ops.rnn_cell_impl.GRUCell object at 0x7fa4d796fe10>
(?, 4) (?, 128)
(1, ?, 128) Tensor("MultiRNNCellZeroState/MultiRNNCellZeroState/zeros:0", shape=(1, 128), dtype=float32)
(1, ?, 128) Tensor("D/rnn/while/Exit_3:0", shape=(1, 128), dtype=float32)
(?, 128)
(?, 2)


In [23]:
model.initial_state[0], model.final_state, model.initial_state, model.is_training, model.actions, \
model.actions_logits, model.loss, model.opt, model.states, model.targetQs

(<tf.Tensor 'strided_slice:0' shape=(128,) dtype=float32>,
 <tf.Tensor 'D/rnn/while/Exit_3:0' shape=(1, 128) dtype=float32>,
 <tf.Tensor 'MultiRNNCellZeroState/MultiRNNCellZeroState/zeros:0' shape=(1, 128) dtype=float32>,
 <tf.Tensor 'is_training:0' shape=() dtype=bool>,
 <tf.Tensor 'actions:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'D/dense_1/BiasAdd:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'Mean:0' shape=() dtype=float32>,
 <tf.Operation 'Adam' type=NoOp>,
 <tf.Tensor 'states:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'targetQs:0' shape=(?,) dtype=float32>)

## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [24]:
def learn(sess, batch, gamma=0.99):
    
    states = np.array([each[0] for each in batch if each is not None])
    actions = np.array([each[1] for each in batch if each is not None])
    next_states = np.array([each[2] for each in batch if each is not None])
    rewards = np.array([each[3] for each in batch if each is not None])
    dones = np.array([each[4] for each in batch if each is not None])
    initial_states = np.array([each[5] for each in batch if each is not None])
    #print('initial_states.shape', initial_states.shape)
    #print('initial_states[0].shape', initial_states[0].shape)

    next_actions_logits = sess.run(model.actions_logits, 
                                   feed_dict = {model.states: next_states,
                                                model.initial_state: initial_states[1],
                                                model.is_training: False})

    nextQs = np.max(next_actions_logits, axis=1)
    targetQs = rewards + (gamma * nextQs * (1-dones))

    loss, _ = sess.run([model.loss, model.opt],
                       feed_dict = {model.states: states, 
                                    model.actions: actions,
                                    model.targetQs: targetQs,
                                    model.initial_state: initial_states[0],    
                                    model.is_training: False})
    return loss

In [25]:
def act(sess, state, initial_state, is_training=False):
    
    action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                          feed_dict = {model.states: state.reshape([1, -1]), 
                                                       model.is_training: is_training,
                                                       model.initial_state: initial_state})
    action = np.argmax(action_logits, axis=1)[0]
    #print('action_logits, np.argmax(action_logits, axis=1)')
    #print(action_logits.shape, np.argmax(action_logits, axis=1).shape)
    return action, final_state

In [26]:
# x = []

In [27]:
# x

In [28]:
# import numpy as np
# np.mean(x)

In [ ]:
import numpy as np
from collections import deque

saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    batch = deque(maxlen=batch_size)
    
    # Training episodes/epochs
    for ep in range(1111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state)

        # Training steps/batches
        while True:
            action, final_state =  act(sess=sess, state=state, initial_state=initial_state)
            
            next_state, reward, done, _ = env.step(action)
            
            batch.append([state, action, next_state, reward, float(done), initial_state])
            
            total_reward += reward
            state = next_state
            initial_state = final_state

            # Training/learning
            #print('len(batch), batch.maxlen', len(batch), batch.maxlen)
            if len(batch) >= batch_size:
                loss = learn(sess=sess, batch=batch, gamma=gamma)
                loss_batch.append(loss)
                
            if done is True:
                break
            
        if len(batch) >= batch_size:
            # Outputing: priting out/Potting
            episode_reward.append(total_reward)

            print('Episode:{}'.format(ep),
                  'meanR:{:.2f}'.format(np.mean(episode_reward)),
                  'R:{:.2f}'.format(total_reward),
                  'loss:{:.2f}'.format(np.mean(loss_batch)))

            # Ploting out
            episode_rewards_list.append([ep, np.mean(episode_reward)])
            rewards_list.append([ep, total_reward])
            loss_list.append([ep, np.mean(loss_batch)])

            # Break episode/epoch loop
            if np.mean(episode_reward) >= 500:
                break
                
        else: print('len(batch) < batch_size', len(batch), batch_size )
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

len(batch) < batch_size 26 128
len(batch) < batch_size 73 128
len(batch) < batch_size 93 128
len(batch) < batch_size 113 128
Episode:4 meanR:15.00 R:15.00 loss:0.99
Episode:5 meanR:16.00 R:17.00 loss:1.07
Episode:6 meanR:13.67 R:9.00 loss:1.17
Episode:7 meanR:12.25 R:8.00 loss:1.23
Episode:8 meanR:11.80 R:10.00 loss:1.40
Episode:9 meanR:11.33 R:9.00 loss:1.82
Episode:10 meanR:11.14 R:10.00 loss:2.80
Episode:11 meanR:11.00 R:10.00 loss:4.47
Episode:12 meanR:10.89 R:10.00 loss:5.76
Episode:13 meanR:10.70 R:9.00 loss:6.18
Episode:14 meanR:10.55 R:9.00 loss:5.65
Episode:15 meanR:10.42 R:9.00 loss:5.96
Episode:16 meanR:10.31 R:9.00 loss:6.24
Episode:17 meanR:10.21 R:9.00 loss:6.46
Episode:18 meanR:10.13 R:9.00 loss:6.62
Episode:19 meanR:10.12 R:10.00 loss:6.61
Episode:20 meanR:10.06 R:9.00 loss:6.43
Episode:21 meanR:10.06 R:10.00 loss:6.35
Episode:22 meanR:10.00 R:9.00 loss:6.27
Episode:23 meanR:9.95 R:9.00 loss:6.22
Episode:24 meanR:9.86 R:8.00 loss:6.23
Episode:25 meanR:9.86 R:10.00 loss:

Episode:197 meanR:84.77 R:19.00 loss:119.85
Episode:198 meanR:84.80 R:17.00 loss:125.92
Episode:199 meanR:84.44 R:22.00 loss:88.29
Episode:200 meanR:85.51 R:130.00 loss:25.15
Episode:201 meanR:85.83 R:48.00 loss:13.08
Episode:202 meanR:86.08 R:40.00 loss:21.65
Episode:203 meanR:87.06 R:115.00 loss:15.09
Episode:204 meanR:88.11 R:117.00 loss:3.02
Episode:205 meanR:89.19 R:122.00 loss:18.61
Episode:206 meanR:88.53 R:42.00 loss:24.47
Episode:207 meanR:88.75 R:38.00 loss:38.52
Episode:208 meanR:90.22 R:158.00 loss:23.99
Episode:209 meanR:90.47 R:37.00 loss:24.50
Episode:210 meanR:91.70 R:140.00 loss:28.94
Episode:211 meanR:92.96 R:144.00 loss:22.73
Episode:212 meanR:94.36 R:163.00 loss:21.00
Episode:213 meanR:95.12 R:106.00 loss:27.51
Episode:214 meanR:97.80 R:286.00 loss:3.11
Episode:215 meanR:101.71 R:403.00 loss:3.77
Episode:216 meanR:106.59 R:500.00 loss:6.94
Episode:217 meanR:111.46 R:500.00 loss:10.78
Episode:218 meanR:116.27 R:500.00 loss:11.41
Episode:219 meanR:116.78 R:110.00 loss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episode/epoch
    for _ in range(10):
        total_reward = 0
        state = env.reset()
        initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
        
        # Steps/batches
        while True:
            env.render()
            action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                    feed_dict = {model.states: state.reshape([1, -1]), 
                                                                 model.initial_state: initial_state})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # At the end of each episode
        print('total_reward:{}'.format(total_reward))

# Close the env
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.